In [27]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "tarun7r/Finance-Llama-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.51it/s]


Given the news summary: 'The Federal Reserve is unlikely to cut rates in 2024.', what should an investor do? (Fed, News, Cuts, Rates, 2024)
Given the news summary: 'The Federal Reserve is unlikely to cut rates in 2024.', what should an investor do?
A. Sell stocks
B. Buy bonds
C. Buy gold
D. Buy stocks
Answer: B


In [11]:
prompt = "You are a financial analyst deciding whether to buy or sell a stock based on headlines. Consider market sentiment, historical impact of product launches, and valuation.Headline: Apple announces a new product.Should you buy or sell Apple stock? If you would buy it, would you buy at the current price or wait for a lower one? What is the minimum price you'd consider buying at, and why?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=500,temperature=0.8,)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You are a financial analyst deciding whether to buy or sell a stock based on headlines. Consider market sentiment, historical impact of product launches, and valuation.Headline: Apple announces a new product.Should you buy or sell Apple stock? If you would buy it, would you buy at the current price or wait for a lower one? What is the minimum price you'd consider buying at, and why? If you'd sell it, would you sell at the current price, or wait for a higher one? What is the maximum price you'd sell at, and why?
Given the headline, I would consider buying Apple stock. Apple's new product announcements have historically led to increases in its stock price. For example, in 2019, Apple launched the iPhone 11, which led to an increase of 7% in its stock price. This historical trend suggests that Apple's new product announcements may lead to a positive impact on its stock price.
In addition, Apple's stock is currently undervalued compared to its historical average. Apple's current price-to-e

In [14]:
import yfinance as yf

ticker = yf.Ticker("NVDA")

income_stmt = ticker.financials
balance_sheet = ticker.balance_sheet
cash_flow = ticker.cashflow
info = ticker.info

In [ ]:
year=income_stmt.columns[0]
EPS=income_stmt[year]["Diluted EPS"]
Net_Income=income_stmt[year]["Net Income"]
Gross_Profit=income_stmt[year]["Gross Profit"]
Revenue=income_stmt[year]["Total Revenue"]
Total_Assets=balance_sheet[year]["Total Assets"]
Total_Liabilities=balance_sheet[year]["Total Liabilities Net Minority Interest"]
Shareholders_Equity=balance_sheet[year]["Stockholders Equity"]
Free_Cash_Flow=cash_flow[year]["Free Cash Flow"]
Investing_Cash_Flow=cash_flow[year]["Investing Cash Flow"]
Financing_Cash_Flow=cash_flow[year]["Financing Cash Flow"]
Operating_Cash_Flow=cash_flow[year]["Operating Cash Flow"]
P_E=ticker.info['trailingPE']
ROA=ticker.info['returnOnAssets']
ROE=ticker.info['returnOnEquity']

In [41]:
def get_financial_info(ticker):
    ticker = yf.Ticker(ticker)
    income_stmt = ticker.financials
    balance_sheet = ticker.balance_sheet
    cash_flow = ticker.cashflow
    info = ticker.info
    year=income_stmt.columns[0]
    EPS=income_stmt[year]["Diluted EPS"]
    Net_Income=income_stmt[year]["Net Income"]
    Gross_Profit=income_stmt[year]["Gross Profit"]
    Revenue=income_stmt[year]["Total Revenue"]
    Total_Assets=balance_sheet[year]["Total Assets"]
    Total_Liabilities=balance_sheet[year]["Total Liabilities Net Minority Interest"]
    Shareholders_Equity=balance_sheet[year]["Stockholders Equity"]
    Free_Cash_Flow=cash_flow[year]["Free Cash Flow"]
    Investing_Cash_Flow=cash_flow[year]["Investing Cash Flow"]
    Financing_Cash_Flow=cash_flow[year]["Financing Cash Flow"]
    Operating_Cash_Flow=cash_flow[year]["Operating Cash Flow"]
    P_E=ticker.info['trailingPE']
    ROA=ticker.info['returnOnAssets']
    ROE=ticker.info['returnOnEquity']

    prompt = f"""
        You are a fundamental investment analyst. Analyze the financial performance of {ticker} and give a recommendation: Strong Buy, Buy, Hold, or Sell. Justify your decision in 4–6 bullet points using financial reasoning.

        Financials for {ticker}:

        Income Statement:
        - Revenue: ${Revenue:,.0f}
        - Gross Profit: ${Gross_Profit:,.0f}
        - Net Income: ${Net_Income:,.0f}
        - EPS (Diluted): {EPS:.2f}

        Balance Sheet:
        - Total Assets: ${Total_Assets:,.0f}
        - Total Liabilities: ${Total_Liabilities:,.0f}
        - Shareholders' Equity: ${Shareholders_Equity:,.0f}

        Cash Flow:
        - Operating Cash Flow: ${Operating_Cash_Flow:,.0f}
        - Free Cash Flow: ${Free_Cash_Flow:,.0f}
        - Investing Cash Flow: ${Investing_Cash_Flow:,.0f}
        - Financing Cash Flow: ${Financing_Cash_Flow:,.0f}

        Valuation and Ratios:
        - P/E Ratio: {P_E:.2f}
        - ROA: {ROA:.2%}
        - ROE: {ROE:.2%}

        Based on this, what is your investment recommendation?
        """
    return prompt

In [45]:
x=get_financial_info("RTX")
x

"\n        You are a fundamental investment analyst. Analyze the financial performance of yfinance.Ticker object <RTX> and give a recommendation: Strong Buy, Buy, Hold, or Sell. Justify your decision in 4–6 bullet points using financial reasoning.\n\n        Financials for yfinance.Ticker object <RTX>:\n\n        Income Statement:\n        - Revenue: $80,738,000,000\n        - Gross Profit: $15,410,000,000\n        - Net Income: $4,774,000,000\n        - EPS (Diluted): 3.55\n\n        Balance Sheet:\n        - Total Assets: $162,861,000,000\n        - Total Liabilities: $100,903,000,000\n        - Shareholders' Equity: $60,156,000,000\n\n        Cash Flow:\n        - Operating Cash Flow: $7,159,000,000\n        - Free Cash Flow: $3,923,000,000\n        - Investing Cash Flow: $-1,534,000,000\n        - Financing Cash Flow: $-6,617,000,000\n\n        Valuation and Ratios:\n        - P/E Ratio: 42.97\n        - ROA: 3.47%\n        - ROE: 7.80%\n\n        Based on this, what is your invest